1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 쪼갠 문서를 임베딩해서 벡터 데이터베이스에 저장한다
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색을 한다
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달한다

# 1. 문서의 내용을 읽는다

In [2]:
# LangChain을 할때는 document loader를 사용했다
# 여기서 document loader를 사용할 수 없기 때문에 python-docx를 사용할 것이다
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-docx] [python-docx]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from docx import Document

# './tax.docx' 경로에 있는 워드 파일을 읽어서
# Document 객체로 로드
document = Document('./tax.docx') 
print(f'document == {dir(document)}')

# 모든 문단(paragraph)의 텍스트를
# 하나로 모아서 저장할 문자열 변수
full_text =''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}')
    full_text += f'{paragraph.text}\n'

# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [15]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    return text_list

In [16]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [19]:
import chromadb
chroma_client = chromadb.Client()

In [20]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [21]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')

In [25]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [26]:
id_list = []
for index in range(len(chunk_list)):
        id_list.append(f'{index}')

In [27]:
len(id_list)

125

In [28]:
len(chunk_list)

125

In [29]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색

In [34]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [35]:
retrieved_doc['documents'][0]

[' 장제비\n마. 「고용보험법」에 따라 받는 실업급여, 육아휴직 급여, 육아기 근로시간 단축 급여, 출산전후휴가 급여등, 「제대군인 지원에 관한 법률」에 따라 받는 전직지원금, 「국가공무원법」ㆍ「지방공무원법」에 따른 공무원 또는 「사립학교교직원 연금법」ㆍ「별정우체국법」을 적용받는 사람이 관련 법령에 따라 받는 육아휴직수당(「사립학교법」 제70조의2에 따라 임명된 사무직원이 학교의 정관 또는 규칙에 따라 지급받는 육아휴직수당으로서 대통령령으로 정하는 금액 이하의 것을 포함한다)\n바. 「국민연금법」에 따라 받는 반환일시금(사망으로 받는 것만 해당한다) 및 사망일시금\n사. 「공무원연금법」, 「공무원 재해보상법」, 「군인연금법」, 「군인 재해보상법」, 「사립학교교직원 연금법」 또는 「별정우체국법」에 따라 받는 공무상요양비ㆍ요양급여ㆍ장해일시금ㆍ비공무상 장해일시금ㆍ비직무상 장해일시금ㆍ장애보상금ㆍ사망조위금ㆍ사망보상금ㆍ유족일시금ㆍ퇴직유족일시금ㆍ유족연금일시금ㆍ퇴직유족연금일시금ㆍ퇴역유족연금일시금ㆍ순직유족연금일시금ㆍ유족연금부가금ㆍ퇴직유족연금부가금ㆍ퇴역유족연금부가금ㆍ유족연금특별부가금ㆍ퇴직유족연금특별부가금ㆍ퇴역유족연금특별부가금ㆍ순직유족보상금ㆍ직무상유족보상금ㆍ위험직무순직유족보상금ㆍ재해부조금ㆍ재난부조금 또는 신체ㆍ정신상의 장해ㆍ질병으로 인한 휴직기간에 받는 급여\n아. 대통령령으로 정하는 학자금\n자. 대통령령으로 정하는 실비변상적(實費辨償的) 성질의 급여\n차. 외국정부(외국의 지방자치단체와 연방국가인 외국의 지방정부를 포함한다. 이하 같다) 또는 대통령령으로 정하는 국제기관에서 근무하는 사람으로서 대통령령으로 정하는 사람이 받는 급여. 다만, 그 외국정부가 그 나라에서 근무하는 우리나라 공무원의 급여에 대하여 소득세를 과세하지 아니하는 경우만 해당한다.\n카. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비\n타. 「전직대통령 예우에 관한 법률」에 따라 받는 연금\n파. 작전임무를 수행하기 위하여 외국에 주

# 5. LLM 질의

In [36]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [37]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [41]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 종합소득세율을 적용해야 합니다. 이는 한국의 소득세 과세표준을 기준으로 합니다. 기본적인 소득세율은 다음과 같습니다 (2023년 기준):\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 4,600만원 이하: 15%\n3. 4,600만원 초과 8,800만원 이하: 24%\n4. 8,800만원 초과 1억5천만원 이하: 35%\n5. 1억5천만원 초과 3억원 이하: 38%\n6. 3억원 초과: 40%\n\n따라서, 연봉 5천만원에 대한 소득세는 다음과 같이 계산됩니다.\n\n1. 1,200만원 * 6% = 72만원\n2. (4,600만원 - 1,200만원) * 15% = 510만원\n3. (5,000만원 - 4,600만원) * 24% = 96만원\n\n위의 금액을 모두 합산하면 다음과 같습니다:\n72만원 + 510만원 + 96만원 = 678만원\n\n하지만, 여기서 끝나는 것이 아니라 근로소득세액공제를 적용해야 합니다. 환산된 총급여액 5천만원에 대한 근로소득세액공제는 다음과 같습니다:\n\n- 3천 300만원 초과 7천만원 이하: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]\n\n따라서:\n74만원 - [(5천만원 - 3천 300만원) × 8/1000] = 60.4만원 (약 60만 4천원)\n\n최종 소득세는:\n678만원 - 60만 4천원 = 약 617만 6천원\n\n따라서, 연봉 5천만원인 직장인의 연간 소득세는 약 617만 6천원입니다. \n\n주의: 이 계산은 근로소득공제 및 기타 소득공제, 세액공제 등을 반영하지 않은 기본적인 계산입니다. 실제 소득세는 개인의 상황에 따라 달라질 수 있습니다.'